In [ ]:
import numpy as np
import pandas as pd
import os

### DataFrame Create

In [ ]:
def make_catndog_dataframe():
    
    paths = []
    dataset_gubuns = []
    label_gubuns = []
    
    for dirname, _, filenames in os.walk('/kaggle/input/cat-and-dog'):
        for filename in filenames:
            if '.jpg' in filename:
                
                file_path = dirname + '/' + filename
                paths.append(file_path)
                
                if '/training_set/' in file_path:
                    dataset_gubuns.append('train')
                elif '/test_set/' in file_path:
                    dataset_gubuns.append('test')
                else:
                    dataset_gubuns.append('N/A')
                
                if 'dogs' in file_path:
                    label_gubuns.append('DOG')
                elif 'cats' in file_path:
                    label_gubuns.append('CAT')
                else:
                    label_gubuns.append('N/A')
    
    data_df = pd.DataFrame({
        'path': paths,
        'dataset': dataset_gubuns,
        'label': label_gubuns
    })
    
    return data_df

In [ ]:
pd.set_option('display.max_colwidth', 200)

data_df = make_catndog_dataframe()

print('data_df shape: ', data_df.shape)

data_df.shape

data_df['dataset'].value_counts(), data_df['label'].value_counts()

### Keras Sequence Dataset

In [ ]:
list_a = [1, 2, 3, 4, 5]

print(len(list_a)) # __len__(self)
print(list_a[0]) # __getitem__(self, index)

In [ ]:
from tensorflow.keras.utils import Sequence
import sklearn
import cv2

BATCH_SIZE = 64
IMAGE_SIZE = 224

In [ ]:
class CnD_Dataset(Sequence):
    
    def __init__(self, image_filenames, labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=False):
        
        self.image_filenames = image_filenames
        self.labels = labels
        self.batch_size = batch_size
        self.augmentor = augmentor
        self.shuffle = shuffle
        
        if self.shuffle:
            pass
    
    def __len__(self):
        return int(np.ceil(len(self.labels)/self.batch_size))
    
    def __getitem__(self, index):
        
        image_name_batch = self.image_filenames[index*self.batch_size:(index+1)*self.batch_size]
        
        if self.labels is not None:
            label_batch = self.labels[index*self.batch_size:(index+1)*self.batch_size]
            
        image_batch = np.zeros((image_name_batch.shape[0], IMAGE_SIZE, IMAGE_SIZE, 3))
        
        for image_index in range(image_name_batch.shape[0]):
            image = cv2.cvtColor(cv2.imread(image_name_batch[image_index]), cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
            
            if self.augmentor is not None:
                image = self.augmentor(image=image)['image']
            
            image_batch[image_index] = image
        
        return image_batch, label_batch
    
    def on_epoch_end(self):
        if (self.shuffle):
            self.image_filenames, self.labels = sklearn.utils.shuffle(self.image_filenames, self.labels)
        else:
            pass

### Keras Sequence Dataset Create

In [ ]:
data_df.head()

In [ ]:
train_df = data_df[data_df['dataset']=='train']
train_df.head()

In [ ]:
train_image_filenames = train_df['path'].values

train_image_filenames.shape

In [ ]:
train_image_labels = train_df['label'].values

train_image_labels.shape

In [ ]:
import albumentations as A

cnd_augmentor = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(p=0.5)
])

cnd_ds = CnD_Dataset(
    train_image_filenames,
    train_image_labels,
    batch_size=BATCH_SIZE,
    augmentor=cnd_augmentor,
    shuffle=False
)

In [ ]:
images_batch = next(iter(cnd_ds))[0]
labels_batch = next(iter(cnd_ds))[1]

print(images_batch.shape)
print(labels_batch.shape)

In [ ]:
print(images_batch[0])
print()
print(labels_batch[0])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def show_grid_images(images_batch, ncols=4, title=None):
    figure, axs = plt.subplots(figsize=(22, 4), nrows=1, ncols=ncols)
    for i in range(ncols):
        axs[i].imshow(np.array(images_batch[i], dtype='int32'))
        axs[i].axis('off')
        axs[i].set_title(title[i])

show_grid_images(images_batch, ncols=6, title='augmented ' + labels_batch)

### CnD_Dataset with scaling

In [ ]:
class CnD_Dataset(Sequence):
    
    def __init__(self, image_filenames, labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=False):
        
        self.image_filenames = image_filenames
        self.labels = labels
        self.batch_size = batch_size
        self.augmentor = augmentor
        self.shuffle = shuffle
        
        if self.shuffle:
            pass
    
    def __len__(self):
        return int(np.ceil(len(self.labels)/self.batch_size))
    
    def __getitem__(self, index):
        
        image_name_batch = self.image_filenames[index*self.batch_size:(index+1)*self.batch_size]
        
        if self.labels is not None:
            label_batch = self.labels[index*self.batch_size:(index+1)*self.batch_size]
            
        image_batch = np.zeros((image_name_batch.shape[0], IMAGE_SIZE, IMAGE_SIZE, 3))
        
        for image_index in range(image_name_batch.shape[0]):
            image = cv2.cvtColor(cv2.imread(image_name_batch[image_index]), cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
            
            if self.augmentor is not None:
                image = self.augmentor(image=image)['image']
            
            image = image/255.0
            
            image_batch[image_index] = image
        
        return image_batch, label_batch
    
    def on_epoch_end(self):
        if (self.shuffle):
            self.image_filenames, self.labels = sklearn.utils.shuffle(self.image_filenames, self.labels)
        else:
            pass

In [ ]:
cnd_ds = CnD_Dataset(train_image_filenames, train_image_labels, batch_size=BATCH_SIZE, augmentor=cnd_augmentor, shuffle=False)

images_batch = next(iter(cnd_ds))[0]
labels_batch = next(iter(cnd_ds))[1]

print(images_batch.shape, images_batch.shape)
print(images_batch[0:2])

### CnD_Dataset with Pretrained preprocessing function

In [ ]:
class CnD_Dataset(Sequence):
    
    def __init__(self, image_filenames, labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=False, pre_func=None):
        
        self.image_filenames = image_filenames
        self.labels = labels
        self.batch_size = batch_size
        self.augmentor = augmentor
        self.pre_func = pre_func
        self.shuffle = shuffle
        
        if self.shuffle:
            pass
    
    def __len__(self):
        return int(np.ceil(len(self.labels)/self.batch_size))
    
    def __getitem__(self, index):
        
        image_name_batch = self.image_filenames[index*self.batch_size:(index+1)*self.batch_size]
        
        if self.labels is not None:
            label_batch = self.labels[index*self.batch_size:(index+1)*self.batch_size]
            
        image_batch = np.zeros((image_name_batch.shape[0], IMAGE_SIZE, IMAGE_SIZE, 3))
        
        for image_index in range(image_name_batch.shape[0]):
            image = cv2.cvtColor(cv2.imread(image_name_batch[image_index]), cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
            
            if self.pre_func is not None:
                image = self.pre_func(image)
            
            if self.augmentor is not None:
                image = self.augmentor(image=image)['image']
            
            image_batch[image_index] = image
        
        return image_batch, label_batch
    
    def on_epoch_end(self):
        if (self.shuffle):
            self.image_filenames, self.labels = sklearn.utils.shuffle(self.image_filenames, self.labels)
        else:
            pass

In [ ]:
from tensorflow.keras.applications.xception import preprocess_input as xcp_preprocess_input

cnd_ds = CnD_Dataset(
    train_image_filenames,
    train_image_labels,
    batch_size=BATCH_SIZE,
    augmentor=cnd_augmentor,
    shuffle=False,
    pre_func=xcp_preprocess_input
)

images_batch = next(iter(cnd_ds))[0]
labels_batch = next(iter(cnd_ds))[1]

print(images_batch.shape, labels_batch.shape)
print(images_batch[0:2])

### CnD_Dataset with custom preprocessing function

In [ ]:
def zero_one_scaler(image):
    return image/255.0

cnd_ds = CnD_Dataset(
    train_image_filenames,
    train_image_labels,
    batch_size=BATCH_SIZE,
    augmentor=cnd_augmentor,
    shuffle=False,
    pre_func=zero_one_scaler
)

images_batch = next(iter(cnd_ds))[0]
labels_batch = next(iter(cnd_ds))[1]

images_batch[0:2]

### Cnd_Dataset with Albumentations Normalize

In [ ]:
cnd_augmentor_normalized = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(p=0.5),
    A.Normalize()
])

cnd_ds = CnD_Dataset(
    train_image_filenames,
    train_image_labels,
    batch_size=BATCH_SIZE,
    augmentor=cnd_augmentor_normalized,
    shuffle=False,
    pre_func=None
)

images_batch = next(iter(cnd_ds))[0]
labels_batch = next(iter(cnd_ds))[1]
images_batch[0]

### Train, Validation data split and label encoding

In [ ]:
train_df['label']

In [ ]:
train_df['label'].shape

In [ ]:
pd.factorize(train_df['label'])

In [ ]:
pd.factorize(train_df['label'])[0]

In [ ]:
pd.factorize(train_df['label'])[0].shape

In [ ]:
pd.get_dummies(train_df['label'])

In [ ]:
pd.get_dummies(train_df['label']).values

In [ ]:
pd.get_dummies(train_df['label']).shape

In [ ]:
train_labels_enc = pd.factorize(train_df['label'])[0]
print(train_labels_enc.shape)
print(train_labels_enc[:5])

In [ ]:
train_labels_ohe = pd.get_dummies(train_df['label']).values
print(train_labels_ohe.shape)
print(train_labels_ohe[:5])

In [ ]:
from sklearn.model_selection import train_test_split

train_df = data_df[data_df['dataset']=='train']
test_df = data_df[data_df['dataset']=='test']

train_path = train_df['path'].values
train_label_enc = pd.factorize(train_df['label'])[0]

tr_path, val_path, tr_label_enc, val_label_enc = \
    train_test_split(train_path, train_label_enc, test_size=0.15, random_state=2021)

print('train path shape: ', tr_path.shape)
print('train label enc shape: ', tr_label_enc.shape)
print('validation path shape: ', val_path.shape)
print('validation label enc shape: ', val_label_enc.shape)

In [ ]:
cnd_augmentor = A.Compose([
    A.HorizontalFlip(p=0.3),
])

In [ ]:
from tensorflow.keras.applications.xception import preprocess_input as xcp_preprocess_input

tr_ds = CnD_Dataset(
    tr_path,
    tr_label_enc,
    batch_size=BATCH_SIZE,
    augmentor=cnd_augmentor,
    shuffle=True,
    pre_func=xcp_preprocess_input
)

val_ds = CnD_Dataset(
    val_path,
    val_label_enc,
    batch_size=BATCH_SIZE,
    augmentor=cnd_augmentor,
    shuffle=True,
    pre_func=xcp_preprocess_input
)

tr_image_batch = next(iter(tr_ds))[0]
print('train image batch shape: ', tr_image_batch.shape)

val_image_batch = next(iter(val_ds))[0]
print('validation image batch shape: ', val_image_batch.shape)

### Model Create

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense , Conv2D , Dropout , Flatten , Activation, MaxPooling2D , GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam , RMSprop 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping , ModelCheckpoint , LearningRateScheduler
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import MobileNetV2

def create_model(model_name='vgg16', verbose=False):
    
    # pretrained model (feature extractor)
    input_tensor = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    
    if model_name == 'vgg16':
        base_model = VGG(
            input_tensor=input_tensor,
            include_top=False,
            weights='imagenet'
        )
    elif model_name == 'resnet50':
        base_model = ResNet50V2(
            input_tensor=input_tensor,
            include_top=False,
            weights='imagenet'
        )
    elif model_name == 'xception':
        base_model = Xception(
            input_tensor=input_tensor,
            include_top=False,
            weights='imagenet'
        )
    elif model_name == 'mobilenet':
        base_model = MobileNetV2(
            input_tensor=input_tensor,
            include_top=False,
            weights='imagenet'
        )
    
    base_model_output = base_model.output
    
    # classification layer
    x = GlobalAveragePooling2D()(base_model_output)
    
    if model_name != 'vgg16':
        x = Dropout(rate=0.5)(x)
    
    x = Dense(50, activation='relu', name='fc1')(x)
    
    output = Dense(1, activation='sigmoid', name='output')(x)
    
    model = Model(inputs=input_tensor, outputs=output)
    
    if verbose:
        model.summary()
    
    return model

In [ ]:
import tensorflow as tf

tf.keras.backend.clear_session()

In [ ]:
model = create_model(model_name='xception')

model.compile(
    optimizer=Adam(0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

rlr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    mode='min',
    verbose=1
)

ely_cb = EarlyStopping(
    monitor='val_loss',
    patience=5,
    mode='min',
    verbose=1
)

### Test Dataset Create and Evaluation

In [ ]:
test_df = data_df[data_df['dataset']=='test']

test_path = test_df['path'].values
test_label_enc = pd.factorize(test_df['label'])[0]

test_ds = CnD_Dataset(
    test_path,
    test_label_enc,
    batch_size=BATCH_SIZE,
    augmentor=None,
    shuffle=False,
    pre_func=xcp_preprocess_input
)

model.evaluate(test_ds)

### MobileNet

In [ ]:
from tensorflow.keras.applications.mobilenet import preprocess_input as mobile_preprocess_input

def get_train_valid_test(data_df):
    # 학습 데이터와 테스트 데이터용 Dataframe 생성. 
    train_df = data_df[data_df['dataset']=='train']
    test_df = data_df[data_df['dataset']=='test']

    # 학습 데이터의 image path와 label을 Numpy array로 변환 및 Label encoding
    train_path = train_df['path'].values
    train_label = pd.factorize(train_df['label'])[0]
    
    test_path = test_df['path'].values
    test_label = pd.factorize(test_df['label'])[0]

    tr_path, val_path, tr_label, val_label = train_test_split(train_path, train_label, test_size=0.15, random_state=2021)
    
    return tr_path, tr_label, val_path, val_label, test_path, test_label
    
    
def do_train_evaluation(data_df, model_name, augmentor, preprocessing_func):
    # 학습/검증/테스트용 이미지 파일 절대경로와 Label encoding 된 데이터 세트 반환
    tr_path, tr_label, val_path, val_label, test_path, test_label = get_train_valid_test(data_df)
    
    # 학습과 검증용 Sequence Dataset 생성. 
    tr_ds = CnD_Dataset(tr_path, tr_label, batch_size=BATCH_SIZE, augmentor=augmentor, 
                          shuffle=True, pre_func=preprocessing_func)
    val_ds = CnD_Dataset(val_path, val_label, batch_size=BATCH_SIZE, augmentor=None, 
                           shuffle=False, pre_func=preprocessing_func)
    
    # 입력된 model_name에 따라 모델 생성. 
    model = create_model(model_name=model_name)
    # 최종 output 출력을 softmax에서 sigmoid로 변환되었으므로 binary_crossentropy로 변환 
    model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])

    # 2번 iteration내에 validation loss가 향상되지 않으면 learning rate을 기존 learning rate * 0.2로 줄임.  
    rlr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, mode='min', verbose=1)
    # 5번 iteration내에 validation loss가 향상되지 않으면 더 이상 학습하지 않고 종료
    ely_cb = EarlyStopping(monitor='val_loss', patience=7, mode='min', verbose=1)

    N_EPOCHS = 15
    # 학습 수행. 
    history = model.fit(tr_ds, epochs=N_EPOCHS, steps_per_epoch=int(np.ceil(tr_path.shape[0]/BATCH_SIZE)), 
                       validation_data=val_ds, validation_steps=int(np.ceil(val_path.shape[0]/BATCH_SIZE)),
                       callbacks=([rlr_cb, ely_cb]), verbose=1)
    
    # 테스트용 Sequence Dataset 생성후 evaluation 수행. 
    test_ds = CnD_Dataset(test_path, test_label, batch_size=BATCH_SIZE, augmentor=None, 
                           shuffle=False, pre_func=preprocessing_func)

    evaluation_result = model.evaluate(test_ds)
    print(evaluation_result)
    return history, evaluation_result

In [ ]:
from tensorflow.keras.applications.mobilenet import preprocess_input as mobile_preprocess_input

tf.keras.backend.clear_session()

cnd_augmentor1 = A.Compose([
    A.HorizontalFlip(p=0.3),
    A.ShiftScaleRotate(p=0.2),
    A.RandomBrightnessContrast(p=0.2)
])

history1, result1 = do_train_evaluation(data_df, 'mobilenet', cnd_augmentor1, mobile_preprocess_input)

In [ ]:
from tensorflow.keras.applications.mobilenet import preprocess_input as mobile_preprocess_input

tf.keras.backend.clear_session()

cnd_augmentor2 = A.Compose([
    A.HorizontalFlip(p=0.3),
    A.ColorJitter(p=0.2),
    A.CLAHE(p=0.2),
    A.Cutout(p=0.2)
])

history2, result2 = do_train_evaluation(data_df, 'mobilenet', cnd_augmentor2, mobile_preprocess_input)

In [ ]:
tf.keras.backend.clear_session()

cnd_augmentor3 = A.Compose([
    A.HorizontalFlip(p=0.3)
])

history3, result3 = do_train_evaluation(data_df, 'mobilenet', cnd_augmentor3, mobile_preprocess_input)